In [1]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

1103663
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [5]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')

print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [6]:
processed_docs =documents['headline_text'].map(preprocess)

processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [7]:
import gensim.corpora as corpora
dictionary = corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [10]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],dictionary[bow_doc_4310[i][0]],bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4015 ("dampen") appears 1 time.


TF-IDF

In [11]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5893154583024485),
 (1, 0.3892866165028569),
 (2, 0.49651921997736453),
 (3, 0.5046106271280878)]


Running LDA using Bag of Words

In [12]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [13]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"warn" + 0.021*"market" + 0.018*"hospit" + 0.017*"school" + 0.016*"health" + 0.015*"high" + 0.015*"share" + 0.015*"minist" + 0.015*"break" + 0.013*"worker"
Topic: 1 
Words: 0.026*"adelaid" + 0.020*"australian" + 0.020*"test" + 0.017*"tasmanian" + 0.017*"win" + 0.016*"open" + 0.014*"melbourn" + 0.011*"vote" + 0.011*"darwin" + 0.010*"parti"
Topic: 2 
Words: 0.023*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.015*"price" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.011*"search" + 0.011*"green"
Topic: 3 
Words: 0.030*"govern" + 0.022*"south" + 0.020*"north" + 0.017*"countri" + 0.015*"rural" + 0.014*"australia" + 0.014*"hour" + 0.013*"west" + 0.013*"indigen" + 0.011*"communiti"
Topic: 4 
Words: 0.022*"die" + 0.020*"live" + 0.018*"donald" + 0.018*"coast" + 0.016*"gold" + 0.015*"lose" + 0.013*"park" + 0.012*"royal" + 0.012*"beat" + 0.011*"head"
Topic: 5 
Words: 0.027*"elect" + 0.021*"sydney" + 0.018*"say" + 0.014*"labor" + 0.013*"time" + 0.013*"c

Running LDA using TF-IDF

In [14]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.010*"final" + 0.010*"crash" + 0.010*"australia" + 0.009*"leagu" + 0.008*"world" + 0.007*"open" + 0.007*"kill" + 0.006*"india" + 0.005*"islam" + 0.005*"festiv"
Topic: 1 Word: 0.010*"victoria" + 0.010*"donald" + 0.009*"farm" + 0.007*"station" + 0.007*"price" + 0.007*"friday" + 0.006*"power" + 0.006*"cattl" + 0.006*"andrew" + 0.005*"anim"
Topic: 2 Word: 0.018*"charg" + 0.017*"polic" + 0.015*"murder" + 0.013*"court" + 0.011*"death" + 0.011*"woman" + 0.010*"alleg" + 0.009*"jail" + 0.009*"drum" + 0.009*"assault"
Topic: 3 Word: 0.016*"coast" + 0.011*"weather" + 0.011*"gold" + 0.011*"miss" + 0.009*"sport" + 0.009*"grandstand" + 0.009*"search" + 0.008*"juli" + 0.008*"season" + 0.007*"septemb"
Topic: 4 Word: 0.017*"market" + 0.016*"news" + 0.011*"share" + 0.007*"climat" + 0.007*"monday" + 0.007*"australian" + 0.006*"bank" + 0.006*"jam" + 0.005*"april" + 0.005*"busi"
Topic: 5 Word: 0.023*"interview" + 0.018*"rural" + 0.010*"royal" + 0.008*"commiss" + 0.007*"john" + 0.007*"michael

Performance evaluation by classifying sample document using LDA Bag of Words model

In [15]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [16]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5218023061752319	 
Topic: 0.027*"kill" + 0.025*"home" + 0.020*"attack" + 0.020*"crash" + 0.017*"turnbul" + 0.015*"road" + 0.015*"protest" + 0.014*"driver" + 0.013*"farm" + 0.013*"polic"

Score: 0.3181477189064026	 
Topic: 0.030*"govern" + 0.022*"south" + 0.020*"north" + 0.017*"countri" + 0.015*"rural" + 0.014*"australia" + 0.014*"hour" + 0.013*"west" + 0.013*"indigen" + 0.011*"communiti"

Score: 0.020008310675621033	 
Topic: 0.023*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.015*"price" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.011*"search" + 0.011*"green"

Score: 0.02000705897808075	 
Topic: 0.022*"warn" + 0.021*"market" + 0.018*"hospit" + 0.017*"school" + 0.016*"health" + 0.015*"high" + 0.015*"share" + 0.015*"minist" + 0.015*"break" + 0.013*"worker"

Score: 0.020006582140922546	 
Topic: 0.027*"elect" + 0.021*"sydney" + 0.018*"say" + 0.014*"labor" + 0.013*"time" + 0.013*"call" + 0.012*"train" + 0.011*"abus" + 0.011*"budget" + 0.010*"polit"

Score:

using LDA TF-IDF model

In [17]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199586272239685	 
Topic: 0.016*"coast" + 0.011*"weather" + 0.011*"gold" + 0.011*"miss" + 0.009*"sport" + 0.009*"grandstand" + 0.009*"search" + 0.008*"juli" + 0.008*"season" + 0.007*"septemb"

Score: 0.02000727690756321	 
Topic: 0.012*"turnbul" + 0.009*"abbott" + 0.009*"marriag" + 0.008*"rugbi" + 0.007*"peter" + 0.007*"malcolm" + 0.006*"june" + 0.006*"smith" + 0.006*"drought" + 0.006*"toni"

Score: 0.02000594139099121	 
Topic: 0.008*"tasmanian" + 0.007*"fund" + 0.007*"budget" + 0.007*"govern" + 0.006*"council" + 0.006*"live" + 0.006*"plead" + 0.006*"christma" + 0.006*"coal" + 0.005*"street"

Score: 0.020005524158477783	 
Topic: 0.023*"trump" + 0.011*"queensland" + 0.007*"thursday" + 0.005*"obama" + 0.005*"detent" + 0.005*"cancer" + 0.005*"crop" + 0.005*"wallabi" + 0.005*"onlin" + 0.005*"harvest"

Score: 0.020004523918032646	 
Topic: 0.010*"victoria" + 0.010*"donald" + 0.009*"farm" + 0.007*"station" + 0.007*"price" + 0.007*"friday" + 0.006*"power" + 0.006*"cattl" + 0.006*"andr

Testing model on unseen document

In [18]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.32110676169395447	 Topic: 0.032*"trump" + 0.028*"australia" + 0.024*"world" + 0.016*"final" + 0.015*"china"
Score: 0.2125992774963379	 Topic: 0.022*"warn" + 0.021*"market" + 0.018*"hospit" + 0.017*"school" + 0.016*"health"
Score: 0.18333426117897034	 Topic: 0.038*"polic" + 0.028*"charg" + 0.026*"court" + 0.020*"murder" + 0.017*"death"
Score: 0.18285787105560303	 Topic: 0.027*"elect" + 0.021*"sydney" + 0.018*"say" + 0.014*"labor" + 0.013*"time"
Score: 0.016689153388142586	 Topic: 0.030*"govern" + 0.022*"south" + 0.020*"north" + 0.017*"countri" + 0.015*"rural"
Score: 0.01668252795934677	 Topic: 0.026*"adelaid" + 0.020*"australian" + 0.020*"test" + 0.017*"tasmanian" + 0.017*"win"
Score: 0.01668252795934677	 Topic: 0.023*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.015*"price" + 0.014*"farmer"
Score: 0.01668252795934677	 Topic: 0.022*"die" + 0.020*"live" + 0.018*"donald" + 0.018*"coast" + 0.016*"gold"
Score: 0.01668252795934677	 Topic: 0.037*"year" + 0.030*"queensland" + 0.022*"c

In [20]:
print('\nPerplexity:', lda_model.log_perplexity(bow_corpus))

from gensim.models import CoherenceModel

coherence_score_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_score_lda.get_coherence()

print('\nCoherence Score:', coherence_score)


Perplexity: -8.813462981092798

Coherence Score: 0.20492387342171892
